In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import datasets, linear_model
from sklearn import metrics
from statsmodels.api import OLS
import statsmodels.api as sm

from scipy import stats

In [3]:
stock_fpath = r'C:\Users\The_Iron_Maiden\Documents\Python Scripts\Smarket.csv'

stock_data = pd.read_csv(stock_fpath)


For each date, we have recorded the percentage returns for each of the five previous trading days, Lag1 through Lag5. We have also recorded

* Volume: the number of shares traded on the previous day, in billions
* Today: the percentage return on the date in question
* Direction: whether the market was Up or Down on this date

In [10]:
stock_data.head(2)

Unnamed: 0  Year   Lag1   Lag2   Lag3   Lag4   Lag5  Volume  Today  \
0           1  2001  0.381 -0.192 -2.624 -1.055  5.010  1.1913  0.959   
1           2  2001  0.959  0.381 -0.192 -2.624 -1.055  1.2965  1.032   

  Direction  
0        Up  
1        Up

In [12]:
stock_data.rename(columns={'Unnamed: 0':'index'}, 
                 inplace=True)
stock_data.set_index('index')

Year   Lag1   Lag2   Lag3   Lag4   Lag5   Volume  Today Direction
index                                                                   
1      2001  0.381 -0.192 -2.624 -1.055  5.010  1.19130  0.959        Up
2      2001  0.959  0.381 -0.192 -2.624 -1.055  1.29650  1.032        Up
3      2001  1.032  0.959  0.381 -0.192 -2.624  1.41120 -0.623      Down
4      2001 -0.623  1.032  0.959  0.381 -0.192  1.27600  0.614        Up
5      2001  0.614 -0.623  1.032  0.959  0.381  1.20570  0.213        Up
...     ...    ...    ...    ...    ...    ...      ...    ...       ...
1246   2005  0.422  0.252 -0.024 -0.584 -0.285  1.88850  0.043        Up
1247   2005  0.043  0.422  0.252 -0.024 -0.584  1.28581 -0.955      Down
1248   2005 -0.955  0.043  0.422  0.252 -0.024  1.54047  0.130        Up
1249   2005  0.130 -0.955  0.043  0.422  0.252  1.42236 -0.298      Down
1250   2005 -0.298  0.130 -0.955  0.043  0.422  1.38254 -0.489      Down

[1250 rows x 9 columns]

In [13]:
stock_data.describe()

index         Year         Lag1         Lag2         Lag3  \
count  1250.000000  1250.000000  1250.000000  1250.000000  1250.000000   
mean    625.500000  2003.016000     0.003834     0.003919     0.001716   
std     360.988227     1.409018     1.136299     1.136280     1.138703   
min       1.000000  2001.000000    -4.922000    -4.922000    -4.922000   
25%     313.250000  2002.000000    -0.639500    -0.639500    -0.640000   
50%     625.500000  2003.000000     0.039000     0.039000     0.038500   
75%     937.750000  2004.000000     0.596750     0.596750     0.596750   
max    1250.000000  2005.000000     5.733000     5.733000     5.733000   

              Lag4        Lag5       Volume        Today  
count  1250.000000  1250.00000  1250.000000  1250.000000  
mean      0.001636     0.00561     1.478305     0.003138  
std       1.138774     1.14755     0.360357     1.136334  
min      -4.922000    -4.92200     0.356070    -4.922000  
25%      -0.640000    -0.64000     1.257400    -0.639500  
50%       0.038500     0.03850     1.422950     0.038500  
75%       0.596750     0.59700     1.641675     0.596750  
max       5.733000     5.73300     3.152470     5.733000

Thecor()function produces a matrix that contains all of the pairwisecorrelations among the predictors in a data set. 

In [14]:
stock_data.corr()

index      Year      Lag1      Lag2      Lag3      Lag4      Lag5  \
index   1.000000  0.979773  0.035415  0.036022  0.038989  0.041437  0.035025   
Year    0.979773  1.000000  0.029700  0.030596  0.033195  0.035689  0.029788   
Lag1    0.035415  0.029700  1.000000 -0.026294 -0.010803 -0.002986 -0.005675   
Lag2    0.036022  0.030596 -0.026294  1.000000 -0.025897 -0.010854 -0.003558   
Lag3    0.038989  0.033195 -0.010803 -0.025897  1.000000 -0.024051 -0.018808   
Lag4    0.041437  0.035689 -0.002986 -0.010854 -0.024051  1.000000 -0.027084   
Lag5    0.035025  0.029788 -0.005675 -0.003558 -0.018808 -0.027084  1.000000   
Volume  0.546348  0.539006  0.040910 -0.043383 -0.041824 -0.048414 -0.022002   
Today   0.035273  0.030095 -0.026155 -0.010250 -0.002448 -0.006900 -0.034860   

          Volume     Today  
index   0.546348  0.035273  
Year    0.539006  0.030095  
Lag1    0.040910 -0.026155  
Lag2   -0.043383 -0.010250  
Lag3   -0.041824 -0.002448  
Lag4   -0.048414 -0.006900  
Lag5   -0.022002 -0.034860  
Volume  1.000000  0.014592  
Today   0.014592  1.000000